In [24]:
!pip install transformers
!pip install tokenizers
!pip install datasets
!pip install sentencepiece

In [25]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
wikiann_dataset = load_dataset("wikiann","id")

In [ ]:
from datasets import load_dataset
indonlu_dataset = load_dataset("indonlu","nergrit")

In [28]:
wikiann_label_names=wikiann_dataset["train"].features[f"ner_tags"].feature.names

In [ ]:
wikiann_label_names

In [67]:
indonlu_label_names=indonlu_dataset["train"].features[f"ner_tags"].feature.names

In [32]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("cahya/distilbert-base-indonesian")

In [ ]:
tokenizer(
    ["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],
    is_split_into_words=True,
)

In [41]:
label_all_tokens=True
def tokenize_and_aligning_labels(examples):
  tokenize_input=tokenizer(examples["tokens"],truncation=True,is_split_into_words=True)
  labels=[]
  all_labels=examples["ner_tags"]

  for i,label in enumerate(all_labels):
    word_id=tokenize_input.word_id(batch_index=1)
    previous_word_index=None
    label_id=[]
    for word_index in word_id:
      if word_index is None:
        label_id.append(--100)
      elif word_index!=previous_word_index:
        label_id.append(label[word_index])
      else:
        label_id.append(label[word_index]  if label_all_tokens else -100)
      previous_word_index=word_index             
    labels.append(label_id)
  tokenize_input["labels"]=labels
  return tokenize_input

In [43]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [47]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [48]:
tokenize_indonlu_dataset=indonlu_dataset.map(tokenize_and_align_labels,batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [62]:
from transformers import DefaultDataCollator

data_collator=DefaultDataCollator(return_tensors="tf")

In [65]:
tokenize_indonlu_dataset["train"]

Dataset({
    features: ['tokens', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1672
})

In [63]:
batch=data_collator([tokenize_indonlu_dataset["train"][0]])
batch["labels"]

<tf.Tensor: shape=(1, 33), dtype=int64, numpy=
array([[-100,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
           5,    5,    5,    5,    5,    5,    5,    5,    5,    5,    5,
           5,    5,    5,    5,    5,    5,    5,    5,    5,    5, -100]])>

In [58]:
from transformers import AutoModel
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
t

In [59]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import create_optimizer
from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 3
num_train_steps = len(tf_train_dataset) * num_train_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(output_dir="bert-finetuned-squad", tokenizer=tokenizer)

# We're going to do validation afterwards, so no validation mid-training
model.fit(tf_train_dataset, callbacks=[callback], epochs=num_train_epochs)